# 쿼리 라우팅 2.
- 리트리버/쿼리 레벨 라우팅 기능
- 대표적 사용새: 요약용 서치엔진 + 일반 본문 시맨틱 서치엔진 나누어서 입력 쿼리 형식이 뭔지에 따라 가변적으로 작동하도록 함.

In [ ]:
!pip install openai llama_index qdrant_client llama-index-vector-stores-qdrant

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.3/309.3 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 

In [ ]:

import nest_asyncio

nest_asyncio.apply()

In [ ]:
# set up OpenAI
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

OpenAI API Key:··········


In [ ]:
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore

import qdrant_client
from qdrant_client import models
client = qdrant_client.QdrantClient(
    url="",
    api_key="",
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small"
)
Settings.llm= OpenAI(temperature=0,model='gpt-4o-mini')

In [ ]:
from llama_index.core import SimpleDirectoryReader

# Paul Graham 에세이 다큐먼트 로딩
documents = SimpleDirectoryReader("/content/pg").load_data()

In [ ]:
# 청크사이즈 조정
Settings.chunk_size = 1024
nodes = Settings.node_parser.get_nodes_from_documents(documents)

In [ ]:
from llama_index.core import StorageContext
vector_store = QdrantVectorStore(client=client, collection_name="routing_exercise")
storage_context = StorageContext.from_defaults(vector_store=vector_store)


In [ ]:
from llama_index.core import SummaryIndex
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext

#Summary Index: 서머리 담는 용도의 인덱스타입
summary_index = SummaryIndex(nodes, storage_context=storage_context)

#VectorStoreIndex: 일반적인 덴스임베딩 + 메타 담는곳
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)

In [ ]:

# as_query_engine의 response_synthesize 방식으로 tree_summarize 사용.
# Tree_summarize: retrieved chunk들을 tree구조로 계층적 summarize하여 결과 response 생성하는 방식
list_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize"
)
vector_query_engine = vector_index.as_query_engine()

In [ ]:
from llama_index.core.tools import QueryEngineTool

# 쿼리 엔진 툴로 양 인덱스 각각 등록 및 설명란에 LLM Selector가 셀렉팅 기준 참고용으로 작성
list_tool = QueryEngineTool.from_defaults(
    query_engine=list_query_engine,
    description=(
        "Useful for summarization questions related to Paul Graham eassy on"
        " What I Worked On."
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from Paul Graham essay on What"
        " I Worked On."
    ),
)

## Pydantic Selector
쿼리엔진 셀렉터

LLM Selector : LLM으로 하여금 쿼리엔진툴 description 보고 어느걸 선택해야 하는지에 대한 최종 분류결과를 JSON으로 내뱉게 하고, 이후 이 JSON 바탕으로 쿼리엔진툴 셀렉팅을 하는 느낌.

Pydantic Selector: raw JSON 대신 OpenAI의 Function Call API 대신 활용해서 셀렉션 테스크 수행.

SingleSelector, MultiSelector 옵션으로 활용 가능

In [ ]:

from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector, LLMMultiSelector
from llama_index.core.selectors import (
    PydanticMultiSelector,
    PydanticSingleSelector,
)

# 쿼리엔진툴들을 묶음으로 상위 쿼리엔진인 라우터쿼리엔진 사용.
query_engine = RouterQueryEngine(
    selector=PydanticSingleSelector.from_defaults(),
    query_engine_tools=[
        list_tool,
        vector_tool,
    ],
)

In [ ]:
# 요약에 관련된 질문
response = query_engine.query("What is the summary of the document?")
print(str(response))

The document chronicles the journey of an individual who transitioned from writing and programming in his youth to exploring artificial intelligence and eventually becoming a successful entrepreneur and investor. It begins with his early experiences with computers and programming languages, particularly Fortran and Lisp, during his school years. After initially pursuing a philosophy degree in college, he shifted his focus to artificial intelligence, inspired by literature and documentaries.

As he navigated through graduate school, he became disillusioned with the state of AI and redirected his efforts towards Lisp programming, ultimately writing a book on the subject. His career took a significant turn when he co-founded Viaweb, an early e-commerce platform, which was later acquired by Yahoo. This experience led him to recognize the potential of web applications and the importance of startup culture.

The narrative also details his personal life, including his passion for painting and

In [ ]:
# 싱글셀렉터가 선택한 쿼리엔진 확인
print(str(response.metadata["selector_result"]))

selections=[SingleSelection(index=0, reason='The question asks for a summary of the document, making the first choice, which is useful for summarization questions, the most relevant.')]


In [ ]:
# 이번엔 구체적인 질문을 날려보기
response = query_engine.query("What did Paul Graham do after RICS?")
print(str(response))

After attending RISD, Paul Graham returned to the US and got a job at a company called Interleaf, which made software for creating documents. He intended to work for a year to save money before returning to RISD the following fall. At Interleaf, he encountered a scripting language that was a dialect of Lisp, which was a significant aspect of his experience there. However, he found himself to be a bad employee due to his lack of interest in learning C and his irresponsible work habits.


In [ ]:
print(str(response.metadata["selector_result"]))

selections=[SingleSelection(index=1, reason="The question asks for specific context regarding Paul Graham's actions after RICS, which aligns with retrieving specific information from the essay.")]


In [ ]:
# LLM 싱글셀렉터
query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        list_tool,
        vector_tool,
    ],
)

In [ ]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

The document chronicles the journey of an individual who transitioned from writing and programming in his youth to exploring artificial intelligence and eventually becoming a successful entrepreneur and investor. It begins with his early experiences with computers and programming languages, particularly Fortran and Lisp, during his school years. After initially pursuing philosophy in college, he shifted his focus to artificial intelligence, inspired by literature and documentaries.

As he navigated through graduate school, he became disillusioned with the state of AI and redirected his efforts towards Lisp programming, leading to the writing of his book "On Lisp." His career took a significant turn when he co-founded Viaweb, an early e-commerce platform, which was later acquired by Yahoo. Following this, he became involved in angel investing and co-founded Y Combinator, a startup accelerator that revolutionized the way startups receive funding and support.

The narrative also touches o

In [ ]:
print(str(response.metadata["selector_result"]))

selections=[SingleSelection(index=0, reason='The question asks for a summary of the document, which aligns with the purpose of choice 1 that is useful for summarization.')]


In [ ]:
response = query_engine.query("What did Paul Graham do after RICS?")
print(str(response))

After attending RISD, Paul Graham returned to the US and got a job at a company called Interleaf, which made software for creating documents. He worked there for a year before planning to return to RISD the following fall.


In [ ]:
print(str(response.metadata["selector_result"]))

selections=[SingleSelection(index=1, reason="The question asks for specific context regarding Paul Graham's actions after RICS, which aligns with retrieving specific information from the essay.")]


- Multi 셀렉터 (인덱스 여러개 참조해서 리트리브 해야될 경우)

In [ ]:
from llama_index.core import SimpleKeywordTableIndex

# 요약, 구체성에 더해, 이번엔 키워드 기반 추출 엔진도 정의해보자
keyword_index = SimpleKeywordTableIndex(nodes, storage_context=storage_context)

keyword_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context using keywords from Paul"
        " Graham essay on What I Worked On."
    ),
)

In [ ]:
query_engine = RouterQueryEngine(
    selector=PydanticMultiSelector.from_defaults(),
    query_engine_tools=[
        list_tool,
        vector_tool,
        keyword_tool,
    ],
)

In [ ]:
# 설명이 같은 두개의 툴들(벡터툴, 키워드툴) 멀티셀렉션하는지 확인
response = query_engine.query(
    "What were noteable events and people from the authors time at Interleaf"
    " and YC?"
)
print(str(response))

During the author's time at Y Combinator (YC), several notable events and individuals emerged. The first batch of startups included prominent founders such as the creators of Reddit, Justin Kan and Emmett Shear, who later went on to establish Twitch, as well as Aaron Swartz, recognized for his contributions to the RSS specification and advocacy for open access. Sam Altman, who became the second president of YC, was also part of this initial group.

A significant development was the establishment of an innovative funding model, which provided $6,000 per founder for a 6% equity stake. This model was seen as fair and better than previous deals the author had experienced. The author noted the benefits of funding startups in batches, which helped reduce the isolation founders often felt and fostered a supportive community among them.

Additionally, the creation of Hacker News (HN) was a key event. Initially designed as a news aggregator for startup founders, it expanded to attract a wider a

In [ ]:

print(str(response.metadata["selector_result"]))

selections=[SingleSelection(index=1, reason="This choice is useful for retrieving specific context related to notable events and people from the author's time at Interleaf and YC."), SingleSelection(index=2, reason='This choice allows for retrieving specific context using keywords, which can help in identifying notable events and people.')]


# Do It Yourself

In [ ]:
!pip install datasets

In [ ]:
# Dataset 로드
from datasets import load_dataset

ds = load_dataset("HAERAE-HUB/KOREAN-WEBTEXT", split='train[:20]')
data = ds.to_pandas()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1284879 [00:00<?, ? examples/s]

In [ ]:
# Document 오브젝트로 변환
from llama_index.core import Document, VectorStoreIndex
docs = []

#Iterative하게 Document 만들기
for i, row in data.iterrows():
    docs.append(Document(
        text=row['text'],
        # extra_info={'title': row['title']}
    ))

In [ ]:
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore

import qdrant_client
from qdrant_client import models
client = qdrant_client.QdrantClient(
    url="",
    api_key="",
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small"
)
Settings.llm= OpenAI(temperature=0,model='gpt-4o-mini')

# 청크사이즈 조정
Settings.chunk_size = 1024

# 노드로 닥스 분할
nodes = Settings.node_parser.get_nodes_from_documents(docs)

In [ ]:
from llama_index.core import SummaryIndex
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.core import SimpleKeywordTableIndex
# 쿼드란트 벡터스토어 컬렉션 생성
vector_store = QdrantVectorStore(client=client, collection_name="routing_exercise2")

# 스토리지 컨텍스트에 백엔드로 쿼드란트 벡터스토어 연결
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
#Summary Index: 서머리 담는 용도의 인덱스타입
summary_index = SummaryIndex(nodes, storage_context=storage_context)

#VectorStoreIndex: 일반적인 덴스임베딩 + 메타 담는곳
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)

#키워드 인덱스
keyword_index = SimpleKeywordTableIndex(nodes, storage_context=storage_context)

In [ ]:
list_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize"
)
vector_query_engine = vector_index.as_query_engine()

keyword_index_query_engine = keyword_index.as_query_engine()

In [ ]:
from llama_index.core.tools import QueryEngineTool


# 쿼리 엔진 툴로 양 인덱스 각각 등록 및 설명란에 LLM Selector가 셀렉팅 기준 참고용으로 작성
list_tool = QueryEngineTool.from_defaults(
    query_engine=list_query_engine,
    description=(
        "Useful for summarization questions"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context"
    ),
)


keyword_tool = QueryEngineTool.from_defaults(
    query_engine=keyword_index_query_engine,
    description=(
        "Useful for retrieving specific context using keywords"
    ),
)

In [ ]:

from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector, LLMMultiSelector
from llama_index.core.selectors import (
    PydanticMultiSelector,
    PydanticSingleSelector,
)
# 라우터쿼리엔진(멀티셀렉터)으로 묶어주기
query_engine = RouterQueryEngine(
    selector=PydanticMultiSelector.from_defaults(),
    query_engine_tools=[
        list_tool,
        vector_tool,
        keyword_tool,
    ],
)

In [ ]:
ds.to_pandas()

,text,source,token_count,__index_level_0__
0,사이트의 판매량에 기반하여 판매량 추이를 반영한 인터파크 도서에서의 독립적인 판매 ...,oscar2201,3348,0
1,“아~아~잊으랴 어찌 우리 이날을 조국의 원수들이 짓밟아 오던 날을~”6·25의 노...,oscar2201,1427,1
2,일러전쟁의 승패를 가른 쓰시마 해전은 세계 최강으로 평가 받는 발틱함대를 괴멸시켰다...,oscar2201,2458,2
3,"재테크 채널 유튜버이자, 「빚부터 갚아라」, 「원트재무설계 소원을 말해봐」 저자인 ...",oscar2201,2838,3
4,"상급자의 범죄와 비리, 부패를 하급자에게 돌리는 것으로 따지면 타의추종을 불허하는 ...",oscar2201,1628,4
5,최근 언론보도에 의하면 이재현 CJ그룹 회장이 지난해 말 두 자녀에게 증여하였던 주...,oscar2201,1366,5
6,"나는 노무현의 시대를 살지 않았다. 그러니까, 나는 이 땅의 생명체로 살아있긴 했지...",oscar2201,2017,6
7,CBRE가 21일 발표한 ‘2021년 2분기 국내 상업용 부동산 시장 보고서’에 따...,oscar2201,1421,7
8,"안녕하세요. 한화솔루션입니다. 지난주, 슬기로운 솔루션 직장생활 2탄에 이어 이번엔...",oscar2201,2143,8
9,캐나다는 3 년 연속 지구상에서 가장 주목할만한 국가로 선포되었습니다. 일반 타이틀...,oscar2201,1104,9


In [ ]:
# 요약 질문해보기
response = query_engine.query(
    "캐나다가 3년 연속 지구상에서 가장 주목할만한 국가로 선포된 이유에 대해서 요약해봐"
)
print(str(response))

캐나다가 3년 연속 지구상에서 가장 주목할 만한 국가로 선포된 이유는 우수한 교육 프로그램, 경이로운 자연환경, 다문화 사회, 그리고 상대적으로 저렴한 생활비를 제공하기 때문입니다. 이러한 요소들은 국제 학생들에게 매력적인 유학 목적지로 자리 잡게 하였으며, 캐나다의 교육 시스템은 안전하고 개방적이며 관용적인 특성을 지니고 있습니다.


In [ ]:
print(str(response.metadata["selector_result"]))

selections=[SingleSelection(index=0, reason='The question asks for a summary of why Canada has been declared the most remarkable country for three consecutive years.')]


In [ ]:
# 질문해보기
response = query_engine.query(
    "발틱함대가 격파된 전쟁 이름이 뭐지?"
)
print(str(response))

일러전쟁입니다.


In [ ]:
print(str(response.metadata["selector_result"]))

selections=[SingleSelection(index=1, reason='The question asks for a specific historical event, which requires retrieving specific context.'), SingleSelection(index=2, reason='The question can be answered by retrieving specific context using keywords related to the Baltic Fleet and the war.')]


In [ ]:
# 질문해보기
response = query_engine.query(
    "낙동강 댐 개수"
)
print(str(response))

낙동강에 세워진 댐은 8개입니다.


In [ ]:
print(str(response.metadata["selector_result"]))

selections=[SingleSelection(index=1, reason='The question asks for specific information about the number of 낙동강 dams, which requires retrieving specific context.'), SingleSelection(index=2, reason='The question can also be approached by using keywords related to 낙동강 and dams to find the relevant information.')]


In [ ]:
response

Response(response='일러전쟁입니다.', source_nodes=[NodeWithScore(node=TextNode(id_='904efd9a-86ea-485a-b64d-e93faae7952a', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0cf94392-df6c-4ce4-a476-55598b8c93c2', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='e6eba36183ec9c2f4206e87cac5f48b43f541b60d909120edc6d803fa00b5ed5'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='e679c5f5-6a01-42e8-901d-be4355bafd49', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='d9030078912466c13ab7d9e423b521709354845612d43440e5db7f0f3dcc4c8e')}, text='일러전쟁의 승패를 가른 쓰시마 해전은 세계 최강으로 평가 받는 발틱함대를 괴멸시켰다. 쓰시마 해전에서 일방적으로 단 몇시간만에 발틱함대를 궤멸시키자 세계는 충격을 받았고, 서양은 일본을 세계의 7대 강국으로 공식인정하는 계기가 되었다. 당시 러시아는 세계에서 영국에 이은 해군 강국으로 해군의 어뢰정 운용과 전술에 있어 최고로 인정 받고 있었으며, 해군 전술 전략에 있어 세계적으로 존경을 받고 있는 마카로프 제독과 같은 전략가가 있었다. 마카로프 제독은 1904년 일러전쟁 발발 직 후 여순항 해전 중 일본 연함 함대를 발견하고 쫓아가다 기뢰에 걸려 파편에 맞아 즉사했다. 그러나 일러전쟁의 해